# Collect Bird data

This notebook is based on the instructions available here:
https://github.com/ubahnverleih/WoBike/blob/master/Bird.md

## Setup

In [ ]:
%run -i path.py
%run -i setup.py

## Functions

In [ ]:
%run -i functions.py

In [ ]:
# get auth Token
# After running this cell, check your email to get the token

body = {"email": "your@email.com"}

headers = {
    'User-Agent': 'Bird/4.119.0(co.bird.Ride; build:3; iOS 14.3.0) Alamofire/5.2.2',
    'Device-Id': 'BC5BDA72-12A2-5DF4-BD11-92ACF8FFC86D', # is a random 16-bit GUID
    'Platform': 'ios',
    'App-Version': '4.119.0',
    'Content-Type': 'application/json'
    }

POST = 'https://api-auth.prod.birdapp.com/api/v1/auth/email'

r = requests.post(url=POST, headers=headers,json=body)

r.status_code

In [ ]:
# get 'access' and 'refresh' codes

headers = {
    'User-Agent': 'Bird/4.119.0(co.bird.Ride; build:3; iOS 14.3.0) Alamofire/5.2.2',
    'Device-Id': 'BC5BDA72-12A2-5DF4-BD11-92ACF8FFC86D', # is a random 16-bit GUID
    'Platform': 'ios',
    'App-Version': '4.119.0',
    'Content-Type': 'application/json'
    }

tkn = {"token": "YOUR_TOKEN"} # replace "YOUR_TOKEN" with the token you received in the email

POST = "https://api-auth.prod.birdapp.com/api/v1/auth/magic-link/use"

r = requests.post(url=POST, headers=headers,json=tkn)

r.status_code

In [ ]:
## access and refresh token
j = r.json()

## Refresh Auth Token

By default bird tokens expire after one day. They can be easily refreshed without having to get another magic link.

In [ ]:
jj=refresh_token(j).json()

## Load the list of coordinates for the requests

In [ ]:
coords = pd.read_csv(PATH['data']+ placeid + "/"+'request_location_centroids_turin.csv')
coords = coords.drop(['Unnamed: 0'],axis=1)

In [ ]:
# create a list of dict: each dict contains the coordinates for a request

coords_list = []

for i in range(len(coords)):
    
    coord_dict = {'latitude': coords.iloc[i]['latitude'], 'longitude': coords.iloc[i]['longitude']}
    
    coords_list.append(coord_dict)

## Collect e-scooter position data

I used the tutorial:
https://www.dataquest.io/blog/python-api-tutorial/

In [ ]:
responses = []

start_time = time.time()

# set the radius for the requests
R = 500 # (seems to be a dummy variable)

# set the duration (in seconds) of data collecting
duration = 13428000

end_time = start_time + duration

start_day = '2021-05-26'
end_day = '2021-10-28'

request_counter = 0 # if you re-start this cell, update this number to avoid overwriting the data

print('Data collection started at: ', time.ctime(start_time))

print('Data collection will end at: ', time.ctime(end_time))

# make the requests
while time.time() < end_time:
    dfs = []
    
    for i in range(len(coords_list)):
        # make the API call
        response = make_a_request(coords_list[i]['latitude'],coords_list[i]['longitude'],R,jj)
        
        # if we get an error, print the response and halt the loop
        if response.status_code != 200:
            print(response.text)
            jj = refresh_token(jj).json()
            response = make_a_request(coords_list[i]['latitude'],coords_list[i]['longitude'],R,jj)
            
        while response.status_code != 200:
            
            print('Status code = ', response.status_code)
            
            print("Wait 20 minutes than try again...")
            for i in range(20):
                time.sleep(60)
                print(20 - i - 1, " minutes to next try.")
                clear_output(wait=True)
            
            response = make_a_request(coords_list[i]['latitude'],coords_list[i]['longitude'],R,jj)
        
        # create a df with the data of the request
        r_json = response.json()
        r_json_bike = r_json['birds']
        r_df = pd.DataFrame(r_json_bike)
        
        # add to the df WHEN was made the request
        t = time.time()        
        r_df['Time'] = t
        
        # add to the df WHERE was made the request
        r_df['Request Latitude'] = coords_list[i]['latitude']
        r_df['Request Longitude'] = coords_list[i]['longitude']
        
        # rearrange the latitude and longitude data
        lats = []
        lons = []

        for k in range(len(r_df)):
            temp_df = pd.DataFrame(r_json['birds'][k])
            lats.append(temp_df.iloc[0]['location'])
            lons.append(temp_df.iloc[1]['location'])
    
        r_df['latitude'] = lats
        r_df['longitude'] = lons
        
        # delete the 'location' column
        r_df=r_df.drop(['location'], axis=1)
        
        #sort the columns of the df
        r_df=r_df[['id','latitude','longitude','code','model','vehicle_class','captive','partner_id','battery_level','estimated_range','area_key','nest_id','bounty_id','Time','Request Latitude','Request Longitude']]
        
        # append df to a list
        dfs.append(r_df)
        
        # print some output so we can see the status
        text_file = open("Updates.txt", "w")
        n = text_file.write('# '+str(i)+' - Time of the last request: ' + str(time.ctime(time.time())) + '-- Request counter: ' + str(request_counter))
        text_file.close()
        
        # append response to a list
        responses.append(response)
        
        # wait 25 seconds to make the following request, so you don't get banned
        time.sleep(25)
    
    # create a single df for each group of requests
    if len(dfs) == len(coords_list):
        df = pd.concat(dfs)
    
        # export the df as a .csv file
        df.to_csv(PATH['data_api']+ placeid + "/"+'TimeSlot'+str(request_counter)+'_Start'+str(start_day)+'_End'+str(end_day)+'.csv')
    
        request_counter = request_counter + 1
    
    else:
        # print an output to inform you that there is some problem
        text_file = open("Updates.txt", "w")
        n = text_file.write('# '+str(i)+' - Time of the last request: ' + str(time.ctime(time.time()))+ 'IS NOT WORKING - current TimeSlot: '+str(request_counter))
        text_file.close()
    
    #between each group of requests, sleep 20 min 
    time.sleep(1200)